In [1]:
import pandas as pd
import numpy as np
import os

# **Dataset : Fashion Product Images**

In [2]:
fashion_data = pd.read_csv('csv/fashion_product_images.csv', delimiter=',')

#Elegimos solo las columnas textuales
fashion_data = fashion_data[['gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'year', 'usage', 'productDisplayName']]

fashion_data.head()

,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


**Ahora concatenamos y creamos los txt's**

In [3]:
"""# Ruta de la carpeta donde se guardarán los archivos de texto
output_folder = 'fashion_txt'

# Crea la carpeta si no existe
os.makedirs(output_folder, exist_ok=True)

# Itera a través de cada fila y guarda en un archivo de texto
for index, row in fashion_data.iterrows():
    # Convierte la fila en una cadena, separando los elementos por un espacio
    row_str = ' '.join(str(item) for item in row)
    
    # Crea un archivo de texto para cada fila en la carpeta específica
    file_path = os.path.join(output_folder, f'doc_{index + 1}.txt')
    with open(file_path, 'w') as file:
        file.write(row_str)"""

"# Ruta de la carpeta donde se guardarán los archivos de texto\noutput_folder = 'fashion_txt'\n\n# Crea la carpeta si no existe\nos.makedirs(output_folder, exist_ok=True)\n\n# Itera a través de cada fila y guarda en un archivo de texto\nfor index, row in fashion_data.iterrows():\n    # Convierte la fila en una cadena, separando los elementos por un espacio\n    row_str = ' '.join(str(item) for item in row)\n    \n    # Crea un archivo de texto para cada fila en la carpeta específica\n    file_path = os.path.join(output_folder, f'doc_{index + 1}.txt')\n    with open(file_path, 'w') as file:\n        file.write(row_str)"

## **Pre-procesamiento**

In [4]:
import nltk 
from nltk.corpus import stopwords #Stopwords
from nltk.stem import SnowballStemmer #Stemming

#Descargamos las stopwords
nltk.download('stopwords')

#Como todo el contenido esta en ingles, las stopwords tambien lo van a estar
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Tokenizamos nuestro txt**

In [5]:
file_content = open("fashion_txt/doc_1.txt", encoding="utf-8").read().lower()
tokens = nltk.word_tokenize(file_content)
print(tokens)

['men', 'apparel', 'topwear', 'shirts', 'navy', 'blue', 'fall', '2011.0', 'casual', 'turtle', 'check', 'men', 'navy', 'blue', 'shirt']


**Eliminamos de nuestro txt los stopwords y los signos innecesarios**

In [6]:
#Filtramos en caso no pertenezca a los stopwords o sea un valor alfanumerico
texto_filtrado = [word for word in tokens if not word in stopwords and word.isalnum()]
print(texto_filtrado)

['men', 'apparel', 'topwear', 'shirts', 'navy', 'blue', 'fall', 'casual', 'turtle', 'check', 'men', 'navy', 'blue', 'shirt']


**Aplicamos stemming en nuestros txt's**

In [7]:
stemmer = SnowballStemmer('english')

texto_filtrado = [stemmer.stem(w) for w in texto_filtrado]

print(texto_filtrado)

['men', 'apparel', 'topwear', 'shirt', 'navi', 'blue', 'fall', 'casual', 'turtl', 'check', 'men', 'navi', 'blue', 'shirt']
